<a href="https://colab.research.google.com/github/Adi1116/Mindcase_AdityaSharma_IITKGP/blob/main/MindCase_AdityaSharma_Assignmnent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pypdf
!pip install torch
!pip install -q transformers
!pip -q install sentence-transformers
!pip install -q llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 193.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 218.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.62-cp310-cp310-manylinux_2_35_x86_64.whl size=38976308 sha256=53910829f494b89964d7b715e6f87357e612753243bda97b9d3c48f0b53392f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-5m2m10ua/wheels/c0/81/de/d4cc8f152d89865379dbf28ca672358c667192ee55deaca7cb
Successfully built llama-cpp-python


In [3]:
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-llama-cpp

In [4]:
!pip install cuda

ERROR: Could not find a version that satisfies the requirement cuda (from versions: none)
ERROR: No matching distribution found for cuda


In [5]:
import torch
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)
# Initializing LlamaCPP with a pretrained model.
llm = LlamaCPP(
    # I provide the option to specify the URL of a GGML model for automatic download.
    # The model_url parameter can be uncommented and filled accordingly.
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf',

    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,

    # I limit the context window to allow for some flexibility.
    context_window=4096,

    # Additional arguments to be passed to the __call__() method.
    generate_kwargs={},

    # Additional arguments to be passed to the __init__() method.
    # Setting n_gpu_layers to -1 enables GPU usage.
    model_kwargs={"n_gpu_layers": -1},

    # Transforming inputs into Llama2 format.
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

total size (MB): 4368.44


4167it [00:30, 135.83it/s]                         
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /tmp/llama_index/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_

In [6]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Document

# Loading data from the specified input file.
documents = SimpleDirectoryReader(
    input_files = ["/content/Mindcase Data.pdf"]
).load_data()

# Combining text from all loaded documents into a single Document object.
documents = Document(text = "\n\n".join([doc.text for doc in documents]))

In [7]:
import os
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import VectorStoreIndex, ServiceContext, load_index_from_storage
from llama_index.core import StorageContext
def get_build_index(documents,llm,embed_model="local:BAAI/bge-small-en-v1.5",sentence_window_size=3,save_dir="./vector_store/index"):
  # Configuring the node parser for extracting sentence windows.
  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "original_text"
  )

  # Setting up the service context with necessary components.
  sentence_context = ServiceContext.from_defaults(
      llm = llm,
      embed_model= embed_model,
      node_parser = node_parser,
  )

  if not os.path.exists(save_dir):
        # If the save directory doesn't exist, create and load the index.
        index = VectorStoreIndex.from_documents(
            [documents], service_context=sentence_context
        )
        index.storage_context.persist(persist_dir=save_dir)
  else:
      # If the save directory exists, load the existing index.
      index = load_index_from_storage(
          StorageContext.from_defaults(persist_dir=save_dir),
          service_context=sentence_context,
      )

  return index

In [8]:
# Obtain the vector index.
vector_index = get_build_index(documents=documents, llm=llm, embed_model="local:BAAI/bge-small-en-v1.5", sentence_window_size=3, save_dir="./vector_store/index")

<ipython-input-7-c7eb83b9671c>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
  # Configuring postprocessors for metadata replacement and sentence reranking.
  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n, model="BAAI/bge-reranker-base"
  )
  # Creating a query engine from the sentence index with specified parameters.
  engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
  )

  return engine

In [10]:
query_engine = get_query_engine(sentence_index=vector_index, similarity_top_k=6, rerank_top_n=2)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

**Example of queries that should be answered:**

- Explain the theme of the movie?
- Who are the characters?
- How many male and female characters are in the movie?
- Does the script pass the Bechdel test?
- What is the role of Deckard in the movie?

In [11]:
import sys

# Define a function to read input with timeout
def input_with_timeout(prompt, timeout):
    print(prompt, end='', flush=True)
    try:
        response = input()
        return response
    except KeyboardInterrupt:
        sys.exit(0)

# Define a flag to indicate whether the loop should continue running.
running = True

while running:
    try:
        # Input a query from the user.
        query = input("Enter your query (Press 'Esc' to exit): ")

        # Check if the input is 'Esc' to exit.
        if query.lower() == 'esc':
            print("Exiting the loop...")
            running = False
            continue

        # Query the engine with the input query.
        response = query_engine.query(query)

        # Print the response.
        print(response)
        print("\n")

    except KeyboardInterrupt:
        # If the user presses Ctrl+C, exit the loop.
        print("\nExiting the loop...")
        running = False

Enter your query (Press 'Esc' to exit): Explain the theme of the movie?



llama_print_timings:        load time =     570.35 ms
llama_print_timings:      sample time =     113.91 ms /   203 runs   (    0.56 ms per token,  1782.09 tokens per second)
llama_print_timings: prompt eval time =     569.75 ms /   372 tokens (    1.53 ms per token,   652.92 tokens per second)
llama_print_timings:        eval time =    4896.92 ms /   202 runs   (   24.24 ms per token,    41.25 tokens per second)
llama_print_timings:       total time =    6138.38 ms /   574 tokens


 Based on the provided context, it appears that the movie explores themes of identity, deception, and morality. The characters in the scene are discussing a past event where K, the main character, had made an error in identifying a target, leading to unintended consequences. Freysa seems to suggest that this was just a piece of a larger puzzle, implying that there may be more complex truths at play.

Anna's reaction to what she sees in K's memory further emphasizes the theme of identity, as she recognizes something but is unsure of its significance. The overall tone suggests that the characters are grappling with their past actions and their implications on their current reality. Additionally, the use of technology, such as Luv's AI and K's memories, adds a layer of complexity to these themes, raising questions about what it means to be human and the role technology plays in shaping our understanding of ourselves and the world around us.


Enter your query (Press 'Esc' to exit): Who ar

Llama.generate: prefix-match hit

llama_print_timings:        load time =     570.35 ms
llama_print_timings:      sample time =      66.33 ms /   122 runs   (    0.54 ms per token,  1839.29 tokens per second)
llama_print_timings: prompt eval time =     388.44 ms /   324 tokens (    1.20 ms per token,   834.11 tokens per second)
llama_print_timings:        eval time =    2937.94 ms /   121 runs   (   24.28 ms per token,    41.19 tokens per second)
llama_print_timings:       total time =    3720.09 ms /   445 tokens


 Based on the given context, there appear to be three main characters mentioned: Jo, K, and an interviewer (who may or may not be a separate character). Jo is a person who seems to have a special connection with another character named K. The interviewer appears to be asking questions about Jo and K, suggesting that they may be observing or interviewing them. There is also a reference to a digital toy horse that animates in Jo's hand. However, it is unclear if the toy horse is a separate character or just an object belonging to Jo.


Enter your query (Press 'Esc' to exit): How many male and female characters are in the movie?


Llama.generate: prefix-match hit

llama_print_timings:        load time =     570.35 ms
llama_print_timings:      sample time =      53.83 ms /   101 runs   (    0.53 ms per token,  1876.45 tokens per second)
llama_print_timings: prompt eval time =     376.43 ms /   327 tokens (    1.15 ms per token,   868.69 tokens per second)
llama_print_timings:        eval time =    2451.88 ms /   100 runs   (   24.52 ms per token,    40.78 tokens per second)
llama_print_timings:       total time =    3187.28 ms /   427 tokens


 Based on the provided context, there are at least eight characters mentioned: K (male), Doxie #1, Doxie #2 (female), Doxie #3 (female), Mariette (female), Punk Doxie (Mariette) (female), The Emmanator (male), and Sapper Morton (male). Therefore, there are five female characters and three male characters in the movie according to the context.


Enter your query (Press 'Esc' to exit): Does the script pass the Bechdel test?


Llama.generate: prefix-match hit

llama_print_timings:        load time =     570.35 ms
llama_print_timings:      sample time =      52.14 ms /    72 runs   (    0.72 ms per token,  1380.92 tokens per second)
llama_print_timings: prompt eval time =     416.37 ms /   268 tokens (    1.55 ms per token,   643.66 tokens per second)
llama_print_timings:        eval time =    1720.70 ms /    71 runs   (   24.24 ms per token,    41.26 tokens per second)
llama_print_timings:       total time =    2510.38 ms /   339 tokens


 Based on the provided context, the script does not explicitly pass the Bechdel test as there is no conversation between two female characters about something other than a man. However, it's important to note that the Bechdel test is just one way to measure gender representation in media and doesn't capture the full complexity of character interactions or relationships.



Exiting the loop...


# Techniques Used

In the work above, I utilized several techniques and libraries to perform various tasks efficiently. Here's an overview of the key techniques used:

1. **LLAMA Index Integration:** The code integrates with the LLAMA Index library to handle document indexing and querying tasks. LLAMA Index provides efficient methods for processing and querying large collections of text data.

2. **PyTorch:** The PyTorch library is employed for handling tensor computations and machine learning tasks. It is used in conjunction with LLAMA Index for certain operations such as embedding generation and similarity calculations.

3. **Sentence Embeddings:** The code leverages pre-trained sentence embeddings to represent textual data in a high-dimensional vector space. These embeddings capture semantic information from text, enabling similarity comparisons and retrieval of relevant documents.

4. **Query Engine:** A query engine is constructed to facilitate user queries on the indexed documents. The engine utilizes similarity calculations and post-processing techniques to retrieve the most relevant results based on the user's input.

5. **Metadata Processing:** Metadata replacement and sentence reranking are applied as post-processing steps to enhance the quality of search results. These techniques involve modifying metadata associated with document nodes and reordering search results based on additional criteria.

By incorporating these techniques, the code demonstrates effective methods for document indexing, retrieval, and semantic search, making it suitable for various information retrieval tasks.
